# Epigenetics-MOOC Answer Classifier 

In [1]:
import pandas as pd 
import json
import numpy as np
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical

## Prepare the data

The goal here is to convert the answers into word vectors. The word vectors will have elements representing words in the total vocabulary. 

The data needs to be preprocessed to use lower case characters. 

### Read the data

Read the answers and the scores from the JSON file.

Assuming the data was put into separate csv files, the calls will be.

Can also read data directly from the JSON file into a dictionary.  

In [2]:
#TODO: Put this in a csv file so we don't have to re-parse the json every time
import load_json

answers, scores = load_json.get_features("../../data/extractedRawDataJSON")

### Get word frequency distribution

Count how often each word appears in the data set. 

Use this count to create a vocabulary to encode the answers and build the word vectors. 

In [43]:
from collections import Counter

word_counts = Counter()

# TODO: Refactor this code to parse the 'answers' object.
# The answers object is defined above.

for idx, answer in enumerate(answers):
        for word in answer.split(" "):
            word_counts[word] += 1

print("Total words in data set: ", len(word_counts))

Total words in data set:  28268


Keep the 10000 most frequent words. 

In [44]:
vocab = sorted(word_counts, key=word_counts.get, reverse=True)[:10000]
print(vocab[:60])

['of', 'the', 'and', 'in', 'to', 'are', 'CpG', 'is', 'methylation', 'DNA', 'islands', 'regions', 'a', 'cancer', 'repetitive', '', 'genes', 'In', 'can', 'intergenic', 'elements', 'gene', 'that', 'at', 'be', 'normal', 'genomic', 'which', 'or', 'as', 'tumor', 'cells', 'these', 'suppressor', 'by', 'with', 'silencing', 'cell', 'The', 'promoters', 'This', 'this', 'methylated', 'hypermethylation', 'hypomethylation', 'for', 'genes.', 'expression', 'genome', 'repeats', 'tumour', 'also', 'activation', 'it', 'cells,', 'promoter', 'found', 'instability', 'cancer,', 'transcription']


For each answer in the data, create a word vector. 

In [45]:
word2idx = {word : i for i, word in enumerate(vocab)}

### Convert Answers to Vectors

This method takes a string of words (answer) as input and returns a vector with word counts. 

In [46]:
def text_to_vector(text):
    vector = np.zeros(len(vocab))
    for w in text.split(' '):
        indx = word2idx.get(w,None)
        if indx == None:
            continue
        else:
            vector[indx] += 1
    return vector

Now, run through the entire data set and convert each answer to a word vector.

In [47]:
word_vectors = np.zeros((len(answers), len(vocab)), dtype=np.int_)
for x, text in enumerate(answers):
    word_vectors[x] = text_to_vector(text)

### Define Train, Validation & Test sets

Split the data into train, validation, and test sets. 

The function `to_categorical` from TFLearn reshapes the target data to use X output units.

In [77]:
records = len(scores)

shuffle = np.arange(records)
np.random.shuffle(shuffle)
test_fraction = 0.9

train_split, test_split = shuffle[:int(records*test_fraction)], shuffle[int(records*test_fraction):]
trainX, trainY = word_vectors[train_split,:], np.array(list( scores[i] for i in train_split ))
testX, testY = word_vectors[test_split,:], list( scores[i] for i in test_split )

print(trainY)
# print(trainY.shape)
trainY.shape = (1,len(train_split))
print(trainY)


[ 12.75    12.875   12.6875 ...,  12.8125  13.875   12.25  ]
[[ 12.75    12.875   12.6875 ...,  12.8125  13.875   12.25  ]]


In [54]:
print(trainY)

[12.375, 10.375, 11.25, 13.0, 12.5625, 11.25, 13.4375, 13.0, 1.75, 10.6875, 13.75, 12.375, 13.6875, 13.625, 13.25, 12.4375, 12.75, 12.3125, 12.375, 13.125, 13.625, 13.4375, 11.875, 11.5, 11.875, 11.75, 14.0, 11.6875, 12.25, 12.25, 13.375, 13.9375, 13.5625, 13.75, 6.875, 13.625, 10.5, 13.875, 8.625, 11.25, 11.875, 11.75, 12.25, 12.125, 14.0, 12.75, 13.5, 11.375, 13.25, 12.375, 9.625, 11.375, 6.375, 10.125, 13.8125, 10.125, 12.75, 11.5, 10.125, 13.375, 13.875, 13.625, 13.375, 12.0625, 13.25, 11.25, 12.4375, 11.9375, 13.25, 6.1875, 11.0625, 13.25, 7.59375, 13.3125, 13.3125, 12.875, 9.875, 13.75, 10.375, 12.1875, 11.375, 13.5, 7.375, 13.75, 12.875, 13.125, 10.125, 10.75, 11.5, 8.625, 11.5, 12.375, 12.125, 10.375, 12.0, 13.75, 11.6875, 12.25, 13.625, 13.25, 13.125, 12.875, 12.125, 12.875, 12.125, 12.125, 11.6875, 11.375, 13.875, 9.625, 12.25, 12.625, 13.375, 13.75, 13.375, 12.6875, 12.875, 11.875, 13.75, 11.59375, 11.375, 9.4375, 6.125, 13.125, 13.8125, 11.625, 13.5, 13.8125, 13.625, 13.5, 

# The network

### Input layer

Must provide the number of input units. For our problem, `n_input_units` is the size of the epigenetics vocabulary. 

Setting the first argument to `None` chooses the default batch size.

```
net = tflearn.input_data([None, n_input_units])
```

### Hidden layers

Add hidden layers with 

```
net = tflearn.fully_connected(net, n_units, activation='ReLU')
```

This adds a fully connected layer where every unit in the previous layer is connected to every unit in this layer. 

Arguments:
`net` the network created with the call to `tflearn.input_data`. This tells the network to use the output of the previous layer as the input to this layer. 
`n_units`: the number of units in the layer.
`activation`: the activation function. 

Add more hidden layers by repeatedly calling `net = tflearn.fully_connected(net, n_units)`.

### Output layer

The last layer you add is used as the output layer. 

Set the number of units to match the target data. In our case the score is a single number. We need only one output unit.

```
net = tflearn.fully_connected(net, 1, activation='ReLU')
```

### Training

To set how you train the network, use 

```
net = tflearn.regression(net, optimizer='sgd', learning_rate=0.1, loss='categorical_crossentropy')
```

Arguments: 

* `optimizer` sets the training method, here stochastic gradient descent
* `learning_rate` is the learning rate
* `loss` determines how the network error is calculated. In this example, with the categorical cross-entropy.

Finally, create the model with `tflearn.DNN(net)`. So it ends up looking something like 

```
net = tflearn.input_data([None, X])                          # Input
net = tflearn.fully_connected(net, 5, activation='ReLU')      # Hidden
net = tflearn.fully_connected(net, 1, activation='ReLU')   # Output
net = tflearn.regression(net, optimizer='sgd', learning_rate=0.1, loss='categorical_crossentropy')
model = tflearn.DNN(net)
```

In [79]:
# Build the Neural Network
def build_net():
    # Reset all parameters and variables. Use it if you are using Jupyter
    tf.reset_default_graph()

    
    # Input layer
    # Set the number of input units to be equal to the size of the epigenetics vocabulary
    n_input_units = len(vocab)
    net = tflearn.input_data([None, n_input_units])
    
    # Hidden layers
    net = tflearn.fully_connected(net, 10, activation='ReLU') 
    net = tflearn.fully_connected(net, 5, activation='ReLU')
    
    # Output
    # Number of units is defined by the structure of the score  
    # Setting it to 1 for first iteration of the application where the score is a float (0-12)
    n_output_units = 1
    
    
    # Network parameters
    # optimizer: the training method, here stochastic gradient descent
    # learning_rate: is the learning rate
    # loss` determines how the network error is calculated. In this example, with the categorical cross-entropy.
    #TODO: select the best activation function to predict scores that have values 0-12.
    net = tflearn.fully_connected(net, n_output_units, activation='ReLU')
    net = tflearn.regression(net, optimizer='sgd', learning_rate=0.1, loss='categorical_crossentropy')
       
    nn = tflearn.DNN(net)
    return nn

## Intialize the Neural Network

`build_net()` builds the model. 

Add arguments if you want to change parameters in the model.

In [71]:
model = build_net()

## Training the network

Now that we've constructed the network, saved as the variable `model`, we can fit it to the data. 

Use the `model.fit` method to train the network. 

`trainX`: training features  
`trainY`: training targets . 
`validation_set=0.1`: reserves 10% of the data set as the validation set. 

In [81]:
# Train the network
model.fit(trainX, trainY, validation_set=0.1, show_metric=True, batch_size=128, n_epoch=50)

IndexError: list index out of range

## Testing

Run the network on the test set to measure its performance. 

In [ ]:
# TODO: Adjust this test
predictions = (np.array(model.predict(testX))[:,0] >= 0.5).astype(np.int_)
test_accuracy = np.mean(predictions == testY[:,0], axis=0)
print("Accuracy: ", test_accuracy)